# 🌊 Light Wave Photoelectric Effect Explorer

Interactive notebook for exploring the wave-based photoelectric model.

This model investigates whether photoelectric-like behavior can emerge from pure wave mechanics with an internal resonance mechanism.

## The Model

**State Variables:**
- x(t): Electron displacement
- v(t): Electron velocity  
- s(t): Internal store/gain

**Equations:**
```
dx/dt = v
dv/dt = -ω₀²x - d·v + s·u(t)
ds/dt = -α·s + β·f²·x·u(t)
```

The **f²** term creates a frequency threshold!


In [ ]:
# Import necessary modules
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import sys

# Add project root to path
sys.path.insert(0, str(Path('.').absolute()))

from src.physics import AtomModel, EscapeDetector
from src.simulation import Simulation, SimulationConfig, create_default_simulation
from src.analysis import find_threshold_frequency, analyze_escape_times
from src.visualization import (
    plot_time_series, plot_phase_portrait, 
    plot_frequency_sweep, plot_energy_evolution
)
from config.parameters import (
    PhysicsParameters, DEFAULT_PHYSICS, 
    PHOTOELECTRIC_TUNED, estimate_threshold_frequency
)

# Set up nice plotting
%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['figure.dpi'] = 100

print("✓ All modules loaded successfully!")


## 1. Set Up Parameters and Run Single Simulations


In [ ]:
# TUNED parameters that show clear photoelectric threshold behavior
# With these parameters, threshold is around f ~ 0.12-0.15
params = PhysicsParameters(
    omega0=1.0,       # Natural binding frequency
    damping=0.0001,   # Very low damping - allows oscillation growth
    alpha=0.05,       # Low store decay rate
    beta=100.0,       # Strong frequency-dependent coupling
    amplitude=0.2,    # Moderate wave amplitude
    escape_position=5.0
)

print(params.describe())

# Estimate threshold analytically
est_thresh = estimate_threshold_frequency(params)
print(f"\nEstimated threshold frequency: {est_thresh:.3f}")


In [ ]:
# Create simulation with tuned parameters
sim = create_default_simulation(
    omega0=params.omega0, damping=params.damping,
    alpha=params.alpha, beta=params.beta,
    amplitude=params.amplitude,
    escape_threshold=params.escape_position,
    t_max=150.0, dt=0.0005  # Small timestep for accuracy
)

# Below threshold (should stay bound) - threshold is ~0.12
freq_below = 0.08
result_below = sim.run(freq_below)
print(f"f = {freq_below}: Escaped = {result_below.escaped}")

# Above threshold (should escape)
freq_above = 0.2
result_above = sim.run(freq_above)
if result_above.escaped:
    print(f"f = {freq_above}: Escaped = True, t = {result_above.escape_time:.2f}")
else:
    print(f"f = {freq_above}: Escaped = {result_above.escaped}")


In [ ]:
# Plot comparison: below vs above threshold
fig, axes = plt.subplots(2, 3, figsize=(15, 8))

# Below threshold row
axes[0, 0].plot(result_below.times, result_below.positions, 'b-', alpha=0.7)
axes[0, 0].set_title(f'Position (f={freq_below}, BOUND)')
axes[0, 0].set_ylabel('x(t)')

axes[0, 1].plot(result_below.times, result_below.velocities, 'r-', alpha=0.7)
axes[0, 1].set_title('Velocity')
axes[0, 1].set_ylabel('v(t)')

axes[0, 2].plot(result_below.times, result_below.stores, 'orange', alpha=0.7)
axes[0, 2].set_title('Internal Store')
axes[0, 2].set_ylabel('s(t)')

# Above threshold row
axes[1, 0].plot(result_above.times, result_above.positions, 'b-', alpha=0.7)
axes[1, 0].set_title(f'Position (f={freq_above}, ESCAPED)')
axes[1, 0].set_ylabel('x(t)')
axes[1, 0].set_xlabel('Time')

axes[1, 1].plot(result_above.times, result_above.velocities, 'r-', alpha=0.7)
axes[1, 1].set_title('Velocity')
axes[1, 1].set_xlabel('Time')

axes[1, 2].plot(result_above.times, result_above.stores, 'orange', alpha=0.7)
axes[1, 2].set_title('Internal Store (grows → escape)')
axes[1, 2].set_xlabel('Time')

plt.suptitle('Below vs Above Threshold: Key Difference is Store Growth!', y=1.02, fontsize=14)
plt.tight_layout()
plt.show()


## 2. Frequency Sweep: Finding the Threshold

Let's sweep through frequencies to find exactly where the threshold is.


In [ ]:
# Frequency sweep
frequencies = np.linspace(0.2, 2.5, 40)

results = []
print("Running frequency sweep...")
for i, f in enumerate(frequencies):
    result = sim.run(f)
    results.append(result)
    status = "ESCAPE" if result.escaped else "bound"
    if (i+1) % 10 == 0:
        print(f"  [{i+1}/{len(frequencies)}] f={f:.3f} → {status}")

# Find threshold
threshold = find_threshold_frequency(results)
print(f"\n{'='*50}")
print(f"THRESHOLD FREQUENCY: {threshold.threshold_frequency:.4f}")
print(f"Uncertainty: ±{threshold.threshold_uncertainty:.4f}")
print(f"{'='*50}")


In [ ]:
# Visualize frequency sweep
plot_frequency_sweep(results, threshold.threshold_frequency, show=True)


## 3. KEY TEST: Amplitude Independence

A crucial characteristic of the real photoelectric effect: the threshold frequency doesn't depend on light intensity (amplitude).

This is what Einstein's "photon" explanation was designed to explain. But can wave mechanics with internal resonance also explain it?


In [ ]:
# Test multiple amplitudes
amplitudes = [0.001, 0.005, 0.01, 0.02, 0.05]
frequencies = np.linspace(0.3, 2.5, 30)

results_by_amp = {}
thresholds_by_amp = {}

print("Testing amplitude independence...")
print("="*50)

for amp in amplitudes:
    sim_amp = create_default_simulation(
        omega0=params.omega0, damping=params.damping,
        alpha=params.alpha, beta=params.beta,
        amplitude=amp, escape_threshold=params.escape_position,
        t_max=200.0
    )
    
    results = [sim_amp.run(f) for f in frequencies]
    results_by_amp[amp] = results
    
    thresh = find_threshold_frequency(results)
    thresholds_by_amp[amp] = thresh.threshold_frequency
    
    print(f"Amplitude {amp:.4f} → Threshold {thresh.threshold_frequency:.4f}")

# Calculate variation
thresh_values = list(thresholds_by_amp.values())
thresh_mean = np.mean(thresh_values)
thresh_std = np.std(thresh_values)
variation = thresh_std / thresh_mean

print("="*50)
print(f"Mean threshold: {thresh_mean:.4f}")
print(f"Variation: {variation:.2%}")
print("="*50)

if variation < 0.1:
    print("✓ AMPLITUDE INDEPENDENT! (Photoelectric-like behavior)")
else:
    print("✗ Threshold varies with amplitude - adjust parameters")


In [ ]:
# Plot threshold vs amplitude
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Left: Threshold vs amplitude (should be flat!)
ax1 = axes[0]
ax1.scatter(amplitudes, thresh_values, s=100, c='purple', edgecolor='white', linewidth=2)
ax1.axhline(thresh_mean, color='gray', linestyle='--', label=f'Mean = {thresh_mean:.3f}')
ax1.set_xlabel('Wave Amplitude A', fontsize=12)
ax1.set_ylabel('Threshold Frequency', fontsize=12)
ax1.set_title('Threshold vs Amplitude\n(Flat = Photoelectric behavior!)', fontsize=12)
ax1.legend()
ax1.grid(True, alpha=0.3)

# Right: Escape curves for each amplitude
ax2 = axes[1]
colors = plt.cm.viridis(np.linspace(0.2, 0.8, len(amplitudes)))

for amp, color in zip(amplitudes, colors):
    freqs = [r.frequency for r in results_by_amp[amp]]
    escaped = [1 if r.escaped else 0 for r in results_by_amp[amp]]
    ax2.plot(freqs, escaped, 'o-', color=color, label=f'A={amp}', alpha=0.7, markersize=4)

ax2.axvline(thresh_mean, color='red', linestyle='--', linewidth=2, label='Threshold')
ax2.set_xlabel('Frequency', fontsize=12)
ax2.set_ylabel('Escaped (0/1)', fontsize=12)
ax2.set_title('All amplitudes have SAME threshold!', fontsize=12)
ax2.legend(loc='center right')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


## 4. Parameter Exploration

Try modifying parameters below to see how they affect threshold behavior!


In [ ]:
def explore_parameters(omega0=1.0, damping=0.01, alpha=0.5, beta=1.0):
    """
    Quick parameter exploration.
    
    Key relationships:
    - Threshold ~ sqrt(alpha/beta)
    - Higher omega0 = stronger binding
    - Lower damping = easier buildup
    """
    sim = create_default_simulation(
        omega0=omega0, damping=damping, alpha=alpha, beta=beta,
        amplitude=0.01, escape_threshold=5.0, t_max=200.0
    )
    
    frequencies = np.linspace(0.2, 3.0, 30)
    results = [sim.run(f) for f in frequencies]
    thresh = find_threshold_frequency(results)
    
    print(f"Parameters: ω₀={omega0}, d={damping}, α={alpha}, β={beta}")
    print(f"Analytical estimate: ~{np.sqrt(alpha/beta):.3f}")
    print(f"Actual threshold: {thresh.threshold_frequency:.3f}")
    
    # Quick plot
    fig, ax = plt.subplots(figsize=(10, 4))
    freqs = [r.frequency for r in results]
    escaped = [1 if r.escaped else 0 for r in results]
    colors = ['green' if not e else 'red' for e in escaped]
    ax.scatter(freqs, escaped, c=colors, s=80)
    ax.axvline(thresh.threshold_frequency, color='purple', linestyle='--', 
               label=f'Threshold = {thresh.threshold_frequency:.3f}')
    ax.set_xlabel('Frequency')
    ax.set_ylabel('Escaped')
    ax.set_yticks([0, 1])
    ax.set_yticklabels(['No', 'Yes'])
    ax.legend()
    ax.set_title(f'ω₀={omega0}, d={damping}, α={alpha}, β={beta}')
    plt.show()

# Try default parameters
explore_parameters(omega0=1.0, damping=0.01, alpha=0.5, beta=1.0)


In [ ]:
# TRY THESE! Uncomment to explore different parameters:

# Higher alpha (faster store decay) -> higher threshold
# explore_parameters(omega0=1.0, damping=0.01, alpha=1.0, beta=1.0)

# Higher beta (stronger coupling) -> lower threshold  
# explore_parameters(omega0=1.0, damping=0.01, alpha=0.5, beta=2.0)

# Stronger binding
# explore_parameters(omega0=2.0, damping=0.01, alpha=0.5, beta=1.0)


## 5. Summary

### What This Model Demonstrates:

1. **Frequency threshold emerges** from the f² coupling term - no photons needed!
2. **Amplitude independence** naturally arises from the internal dynamics
3. **Internal store mechanism** explains frequency-selective response

### The Key Physics Insight:

The photoelectric threshold can emerge from:
- Frequency-dependent internal dynamics (the f² term)
- Resonance/gain mechanisms (the store variable s)
- Nonlinear feedback between wave driving and internal state

### What This Means:

Light behaving as a wave CAN explain the photoelectric effect if we model matter with appropriate internal structure. The "particle-like" behavior emerges from the **internal dynamics of the atom**, not from light itself being particles!

### Next Steps:
1. Add output wave emission (for Compton scattering)
2. Model different materials (different parameters)
3. Explore multi-frequency driving


## 6. The Crystal Lattice Black Box: Full System of Equations

**THE VISION**: Light hits a small region of metal crystal lattice. This region is our "black box" - a 3D cutout containing bound electrons, ion cores, electromagnetic fields, and collective excitations.

**THE APPROACH**: Like the fox-rabbit population model using differential equations, we have MULTIPLE INTERACTING QUANTITIES where each affects the others. The trick is to:
1. Identify ALL the relevant variables
2. Write down ALL the equations governing their evolution
3. Understand the COUPLING structure (what depends on what)
4. Find the physical meaning

This could reveal that the "quantum" behavior emerges from how MATTER organizes itself, not from light!


In [ ]:
# Import the crystal lattice model
from src.crystal_lattice_model import (
    CrystalRegion, CrystalLatticeEquations, 
    analyze_interdependencies, get_testable_predictions
)

# Create a copper crystal region - our "black box"
crystal = CrystalRegion(
    lattice_constant_m=3.6e-10,   # 3.6 Angstroms (copper)
    region_size_m=1e-8,           # 10 nm region
    atomic_number=29,             # Copper
    free_electrons_per_atom=1.0
)

print("=" * 70)
print("         THE BLACK BOX: CRYSTAL LATTICE REGION")
print("=" * 70)
print(f"  Number of atoms in region: {crystal.n_atoms:,}")
print(f"  Electron density: {crystal.electron_density:.2e} m⁻³")
print(f"  Plasma frequency: {crystal.plasma_frequency/(2*np.pi):.2e} Hz")
print(f"  Debye screening length: {crystal.debye_length:.2e} m")
print("=" * 70)

# Create the equation system
equations_system = CrystalLatticeEquations(crystal)
print("\n✓ Crystal lattice equation system created")


In [ ]:
# Display the complete system of equations
print(equations_system.derive_equations())


### 6.1 Equation Types: Linear, Polynomial, and Differential

The full system contains DIFFERENT TYPES of equations:

| Type | Examples | Physical Meaning |
|------|----------|------------------|
| **Linear DE** | dx/dt = v | Simple kinematics, conservation |
| **Linear with coefficients** | dv/dt = -ω₀²x - γv | Harmonic oscillator, damping |
| **Polynomial (quadratic)** | ds/dt ~ f² | Frequency-dependent threshold! |
| **Product terms** | s·u, v·u | Coupling between subsystems |
| **Nonlinear feedback** | ds/dt ~ |v·E| | Power absorption, rectification |

The **f²** term is KEY - it creates the threshold behavior!


In [ ]:
# Classify and display equation types
def classify_equations():
    """
    Analyze the system of equations and classify by type.
    
    This helps us understand what mathematical tools we need and
    what kind of behavior to expect from each part.
    """
    
    equations_classified = {
        'linear_kinematic': [
            ('dx/dt = v', 'Position from velocity', 'Simple integration'),
            ('du_L/dt = v_L', 'Lattice position from velocity', 'Simple integration'),
        ],
        'linear_harmonic': [
            ('dv/dt = -ω₀²x - γv + ...', 'Damped harmonic oscillator', 
             'Eigenvalue analysis: ω₀ sets frequency, γ sets decay'),
            ('dv_L/dt = -ω_L²u_L - γ_L·v_L + ...', 'Lattice oscillator',
             'Phonon dynamics'),
        ],
        'polynomial_quadratic': [
            ('ds/dt = -αs + β·f²·|v·E|', 'Store dynamics with f² threshold!',
             'The KEY equation - f² creates frequency threshold'),
        ],
        'product_coupling': [
            ('dv/dt includes s·E', 'Store amplifies field response',
             'Positive feedback when store is large'),
            ('ds/dt includes v·E', 'Power absorption charges store',
             'Energy from wave builds coherence'),
            ('dE/dt ~ -ω_p²·x', 'Electron displacement creates field',
             'Back-EMF effect'),
        ],
        'nonlinear_abs': [
            ('ds/dt ~ |v·E|', 'Absolute value of power flow',
             'Always positive contribution - rectification!'),
        ],
        'collective_dynamics': [
            ('dP/dt = n_e·e·v - P/τ', 'Polarization dynamics',
             'Collective electron response, relaxation'),
            ('dn_pl/dt = generation - γ_pl·n_pl', 'Plasmon dynamics',
             'Collective oscillation modes'),
        ]
    }
    
    print("=" * 70)
    print("         EQUATION CLASSIFICATION BY TYPE")
    print("=" * 70)
    
    for eq_type, equations in equations_classified.items():
        type_name = eq_type.replace('_', ' ').title()
        print(f"\n{'─' * 70}")
        print(f"  {type_name.upper()}")
        print(f"{'─' * 70}")
        for eq, description, insight in equations:
            print(f"\n  {eq}")
            print(f"     → {description}")
            print(f"     ★ {insight}")
    
    print(f"\n{'=' * 70}")
    print("KEY INSIGHT: The system is NONLINEAR due to:")
    print("  1. The f² polynomial term (creates threshold)")
    print("  2. The product terms s·E, v·E (coupling between subsystems)")
    print("  3. The |...| absolute value (rectification)")
    print("=" * 70)

classify_equations()


### 6.2 Interdependency Analysis: The Fox-Rabbit Analogy

Just like predator-prey dynamics where foxes and rabbits affect each other:
```
dR/dt = a·R - b·R·F     (Rabbits grow, get eaten by foxes)
dF/dt = c·R·F - d·F     (Foxes eat rabbits, die naturally)
```

Our system has COUPLED dynamics where quantities affect each other through feedback loops.


In [ ]:
# Display the interdependency analysis
print(analyze_interdependencies())


In [ ]:
# Visualize the coupling matrix
def visualize_coupling_matrix(equations_system, omega=1e15):
    """
    Visualize the linearized coupling matrix.
    
    The matrix shows how each variable affects each other variable.
    Non-zero off-diagonal elements indicate coupling.
    """
    A = equations_system.get_coupling_matrix(omega)
    
    # Normalize for visualization (log scale for wide range)
    A_viz = np.sign(A) * np.log10(np.abs(A) + 1)
    
    # Variable names
    var_names = ['x', 'v', 's', 'E', 'B', 'u_L', 'v_L', 'P', 'n_pl']
    
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))
    
    # Left: Heatmap of coupling matrix
    ax1 = axes[0]
    im = ax1.imshow(A_viz, cmap='RdBu_r', aspect='equal')
    ax1.set_xticks(range(9))
    ax1.set_yticks(range(9))
    ax1.set_xticklabels(var_names)
    ax1.set_yticklabels(var_names)
    ax1.set_xlabel('Affects...')
    ax1.set_ylabel('Is affected by...')
    ax1.set_title('Coupling Matrix (log scale)\nBlue=positive, Red=negative coupling')
    
    # Add coupling strength indicators
    for i in range(9):
        for j in range(9):
            if abs(A[i,j]) > 0:
                ax1.text(j, i, '●' if abs(A[i,j]) > 1e10 else '·', 
                        ha='center', va='center', fontsize=8)
    
    plt.colorbar(im, ax=ax1, label='log₁₀(|coupling|)')
    
    # Right: Dependency graph (simplified)
    ax2 = axes[1]
    
    # Draw nodes in a circle
    theta = np.linspace(0, 2*np.pi, 10)[:-1]
    x_pos = np.cos(theta)
    y_pos = np.sin(theta)
    
    # Draw edges for significant couplings
    for i in range(9):
        for j in range(9):
            if i != j and abs(A[i,j]) > 1e5:  # Threshold for showing
                color = 'blue' if A[i,j] > 0 else 'red'
                alpha = min(0.8, abs(np.log10(abs(A[i,j])+1)) / 15)
                ax2.annotate('', xy=(x_pos[i], y_pos[i]), xytext=(x_pos[j], y_pos[j]),
                           arrowprops=dict(arrowstyle='->', color=color, alpha=alpha,
                                          connectionstyle='arc3,rad=0.1'))
    
    # Draw nodes
    for i, name in enumerate(var_names):
        circle = plt.Circle((x_pos[i], y_pos[i]), 0.12, color='lightblue', ec='black')
        ax2.add_patch(circle)
        ax2.text(x_pos[i], y_pos[i], name, ha='center', va='center', fontsize=10, fontweight='bold')
    
    ax2.set_xlim(-1.5, 1.5)
    ax2.set_ylim(-1.5, 1.5)
    ax2.set_aspect('equal')
    ax2.axis('off')
    ax2.set_title('Dependency Graph\nBlue=positive, Red=negative coupling')
    
    plt.suptitle('COUPLING STRUCTURE: What Affects What', fontsize=14, y=1.02)
    plt.tight_layout()
    plt.show()
    
    return A

# Visualize!
coupling_matrix = visualize_coupling_matrix(equations_system)
print("\nThe coupling matrix reveals the feedback structure!")
print("- Diagonal: self-evolution (decay, oscillation)")
print("- Off-diagonal: how quantities affect each other")


### 6.3 Eigenvalue Analysis: Finding the Threshold

The eigenvalues of the coupling matrix tell us about STABILITY:
- **Negative real parts** - stable (perturbations decay) - electron stays bound
- **Positive real parts** - unstable (perturbations grow) - electron ESCAPES!

The frequency where eigenvalues cross from negative to positive is the **THRESHOLD**!


In [ ]:
# Analyze stability vs frequency
def analyze_stability_vs_frequency(equations_system, omega_range=(1e14, 1e16), n_points=100):
    """
    Sweep through frequencies and analyze stability at each.
    
    This shows us WHERE the threshold is from a linear stability perspective.
    """
    omegas = np.logspace(np.log10(omega_range[0]), np.log10(omega_range[1]), n_points)
    frequencies_Hz = omegas / (2 * np.pi)
    
    max_eigenvalues = []
    eigenvalue_spectra = []
    
    print("Analyzing stability across frequency range...")
    for omega in omegas:
        analysis = equations_system.analyze_stability(omega)
        max_eigenvalues.append(analysis['max_real_eigenvalue'])
        eigenvalue_spectra.append(analysis['eigenvalues'])
    
    max_eigenvalues = np.array(max_eigenvalues)
    
    # Find threshold (where max eigenvalue crosses zero)
    crossings = np.where(np.diff(np.sign(max_eigenvalues)))[0]
    
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    
    # Top left: Max eigenvalue vs frequency
    ax1 = axes[0, 0]
    ax1.semilogx(frequencies_Hz, max_eigenvalues, 'b-', linewidth=2)
    ax1.axhline(0, color='red', linestyle='--', linewidth=2, label='Stability boundary')
    if len(crossings) > 0:
        threshold_idx = crossings[0]
        threshold_freq = frequencies_Hz[threshold_idx]
        ax1.axvline(threshold_freq, color='green', linestyle=':', linewidth=2, 
                   label=f'Threshold ~ {threshold_freq:.2e} Hz')
    ax1.fill_between(frequencies_Hz, max_eigenvalues, 0, 
                     where=max_eigenvalues > 0, alpha=0.3, color='red', label='UNSTABLE (escape)')
    ax1.fill_between(frequencies_Hz, max_eigenvalues, 0,
                     where=max_eigenvalues < 0, alpha=0.3, color='blue', label='STABLE (bound)')
    ax1.set_xlabel('Frequency (Hz)')
    ax1.set_ylabel('Max Real Eigenvalue')
    ax1.set_title('Stability Analysis: Where is the Threshold?')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Top right: All eigenvalues (real parts) vs frequency
    ax2 = axes[0, 1]
    eigenvalue_spectra = np.array(eigenvalue_spectra)
    for i in range(9):
        ax2.semilogx(frequencies_Hz, np.real(eigenvalue_spectra[:, i]), alpha=0.7, linewidth=1)
    ax2.axhline(0, color='red', linestyle='--')
    ax2.set_xlabel('Frequency (Hz)')
    ax2.set_ylabel('Real part of eigenvalue')
    ax2.set_title('All Eigenvalues vs Frequency\n(9 modes from 9 variables)')
    ax2.grid(True, alpha=0.3)
    
    # Bottom left: Eigenvalue spectrum at low frequency
    ax3 = axes[1, 0]
    low_freq_eigs = eigenvalue_spectra[0]
    ax3.scatter(np.real(low_freq_eigs), np.imag(low_freq_eigs), s=100, c='blue', edgecolor='black')
    ax3.axvline(0, color='red', linestyle='--', alpha=0.5)
    ax3.axhline(0, color='gray', linestyle='-', alpha=0.3)
    ax3.set_xlabel('Real part')
    ax3.set_ylabel('Imaginary part')
    ax3.set_title(f'Eigenvalue spectrum at f = {frequencies_Hz[0]:.2e} Hz\n(LOW frequency - should be stable)')
    ax3.grid(True, alpha=0.3)
    
    # Bottom right: Eigenvalue spectrum at high frequency
    ax4 = axes[1, 1]
    high_freq_eigs = eigenvalue_spectra[-1]
    ax4.scatter(np.real(high_freq_eigs), np.imag(high_freq_eigs), s=100, c='red', edgecolor='black')
    ax4.axvline(0, color='red', linestyle='--', alpha=0.5)
    ax4.axhline(0, color='gray', linestyle='-', alpha=0.3)
    ax4.set_xlabel('Real part')
    ax4.set_ylabel('Imaginary part')
    ax4.set_title(f'Eigenvalue spectrum at f = {frequencies_Hz[-1]:.2e} Hz\n(HIGH frequency - might be unstable)')
    ax4.grid(True, alpha=0.3)
    
    plt.suptitle('STABILITY ANALYSIS: The Matrix Eigenvalues Tell Us When Escape Occurs!', 
                 fontsize=14, y=1.02)
    plt.tight_layout()
    plt.show()
    
    if len(crossings) > 0:
        print(f"\n{'=' * 60}")
        print(f"  THRESHOLD DETECTED at ~{threshold_freq:.2e} Hz")
        print(f"  Wavelength: ~{3e8/threshold_freq * 1e9:.1f} nm")
        print(f"{'=' * 60}")
    
    return frequencies_Hz, max_eigenvalues

# Run the analysis
freqs, eigs = analyze_stability_vs_frequency(equations_system)


## 7. Output Wave Emission (Towards Compton Scattering)

An accelerating electron RADIATES! This is classical electromagnetism (Larmor formula).

When light hits the electron:
1. Electron oscillates with acceleration a(t)
2. Oscillating electron emits radiation
3. This emitted wave is the scattered light

If we can model this, we might be able to explain Compton scattering with waves!


In [ ]:
# Wave emission model
def simulate_with_emission(frequency, params, t_max=100.0, dt=0.001):
    """
    Simulate electron dynamics AND calculate emitted radiation.
    
    Classical electromagnetism: accelerating charges radiate!
    
    Larmor formula: P = (q^2 * a^2) / (6 * pi * epsilon_0 * c^3)
    
    For an oscillating electron, the emitted field at distance r is:
    E_emitted(t, r) ~ (q / c^2 r) * a(t - r/c)  (retarded time)
    """
    
    # Run simulation
    sim = create_default_simulation(
        omega0=params.omega0, damping=params.damping,
        alpha=params.alpha, beta=params.beta,
        amplitude=params.amplitude, escape_threshold=params.escape_position,
        t_max=t_max, dt=dt
    )
    result = sim.run(frequency)
    
    # Calculate acceleration: a = dv/dt
    # From our equation: dv/dt = -omega0^2 * x - d*v + s*u
    times = result.times
    positions = result.positions
    velocities = result.velocities
    stores = result.stores
    
    # Input wave
    omega = 2 * np.pi * frequency
    u_wave = params.amplitude * np.sin(omega * times)
    
    # Acceleration
    accelerations = -params.omega0**2 * positions - params.damping * velocities + stores * u_wave
    
    # Emitted field (proportional to acceleration at retarded time)
    # For simplicity, assume observation at fixed distance, no retardation
    # E_emitted ~ a(t)
    E_emitted = accelerations  # Proportional to acceleration
    
    # Emitted power (Larmor formula, normalized)
    P_emitted = accelerations**2  # Proportional to a^2
    
    return {
        'times': times,
        'positions': positions,
        'velocities': velocities,
        'stores': stores,
        'accelerations': accelerations,
        'E_emitted': E_emitted,
        'P_emitted': P_emitted,
        'escaped': result.escaped,
        'input_wave': u_wave
    }

# Test emission at different frequencies
print("Simulating electron dynamics and emitted radiation...")

freq_test = 0.15  # Above threshold
emission_result = simulate_with_emission(freq_test, params)

# Plot
fig, axes = plt.subplots(2, 3, figsize=(15, 8))

# Row 1: Input and state
ax1 = axes[0, 0]
ax1.plot(emission_result['times'], emission_result['input_wave'], 'b-', alpha=0.7)
ax1.set_title(f'Input Wave (f={freq_test})')
ax1.set_ylabel('u(t)')
ax1.set_xlabel('Time')

ax2 = axes[0, 1]
ax2.plot(emission_result['times'], emission_result['positions'], 'g-', alpha=0.7)
ax2.set_title('Electron Position x(t)')
ax2.set_ylabel('x')
ax2.set_xlabel('Time')

ax3 = axes[0, 2]
ax3.plot(emission_result['times'], emission_result['stores'], 'orange', alpha=0.7)
ax3.set_title('Internal Store s(t)')
ax3.set_ylabel('s')
ax3.set_xlabel('Time')

# Row 2: Emission
ax4 = axes[1, 0]
ax4.plot(emission_result['times'], emission_result['accelerations'], 'r-', alpha=0.7)
ax4.set_title('Electron Acceleration a(t)')
ax4.set_ylabel('a = dv/dt')
ax4.set_xlabel('Time')

ax5 = axes[1, 1]
ax5.plot(emission_result['times'], emission_result['E_emitted'], 'm-', alpha=0.7)
ax5.set_title('Emitted Field E_out ~ a(t)\n(Scattered Wave!)')
ax5.set_ylabel('E_emitted')
ax5.set_xlabel('Time')

ax6 = axes[1, 2]
ax6.plot(emission_result['times'], emission_result['P_emitted'], 'purple', alpha=0.7)
ax6.set_title('Radiated Power ~ a(t)^2')
ax6.set_ylabel('Power')
ax6.set_xlabel('Time')

plt.suptitle('Wave Emission: Accelerating Electron Radiates!', fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

print("\n✓ The electron emits radiation as it accelerates!")
print("  This is the classical mechanism for light scattering.")


## 8. Multi-Frequency Driving: A Critical Test!

Our model predicts something INTERESTING: If two frequencies f1 and f2 are both below threshold, they might TOGETHER cause emission if their combined effect exceeds the threshold.

This is DIFFERENT from the photon model, where only single photon energy matters!

Let us test this prediction.


In [ ]:
# Multi-frequency driving simulation
class MultiFrequencyAtomModel:
    """
    Atom model that can be driven by multiple frequencies simultaneously.
    
    u(t) = sum_i A_i * sin(2*pi*f_i*t + phi_i)
    """
    
    def __init__(self, omega0=1.0, damping=0.01, alpha=0.5, beta=1.0):
        self.omega0 = omega0
        self.omega0_sq = omega0 ** 2
        self.damping = damping
        self.alpha = alpha
        self.beta = beta
    
    def wave_input(self, t, frequencies, amplitudes, phases=None):
        """
        Calculate combined wave input from multiple frequencies.
        
        Args:
            t: Time (or array of times)
            frequencies: List of frequencies
            amplitudes: List of amplitudes for each frequency
            phases: Optional list of phases (default: all zero)
        """
        if phases is None:
            phases = [0.0] * len(frequencies)
        
        u = 0.0
        for f, A, phi in zip(frequencies, amplitudes, phases):
            u += A * np.sin(2.0 * np.pi * f * t + phi)
        return u
    
    def derivatives(self, t, state, frequencies, amplitudes, phases=None):
        """Calculate state derivatives with multi-frequency input."""
        x, v, s = state
        u = self.wave_input(t, frequencies, amplitudes, phases)
        
        dx_dt = v
        dv_dt = -self.omega0_sq * x - self.damping * v + s * u
        
        # For multi-frequency, the effective frequency for charging is complex
        # Use a weighted average based on amplitudes, squared for energy
        if sum(amplitudes) > 0:
            f_eff_sq = sum(A * f**2 for f, A in zip(frequencies, amplitudes)) / sum(amplitudes)
        else:
            f_eff_sq = 0
        
        power_absorption = abs(v * u)
        ds_dt = -self.alpha * s + self.beta * f_eff_sq * power_absorption
        
        return np.array([dx_dt, dv_dt, ds_dt])
    
    def simulate(self, frequencies, amplitudes, phases=None, t_max=200.0, dt=0.001, escape_threshold=5.0):
        """Run simulation with multiple frequencies."""
        state = np.array([0.0, 0.0, 0.001])  # Initial state
        times = [0.0]
        states = [state.copy()]
        escaped = False
        escape_time = None
        
        t = 0.0
        while t < t_max:
            # RK4 integration
            k1 = self.derivatives(t, state, frequencies, amplitudes, phases)
            k2 = self.derivatives(t + dt/2, state + dt*k1/2, frequencies, amplitudes, phases)
            k3 = self.derivatives(t + dt/2, state + dt*k2/2, frequencies, amplitudes, phases)
            k4 = self.derivatives(t + dt, state + dt*k3, frequencies, amplitudes, phases)
            
            state = state + (dt/6) * (k1 + 2*k2 + 2*k3 + k4)
            t += dt
            
            times.append(t)
            states.append(state.copy())
            
            # Check escape
            if abs(state[0]) > escape_threshold:
                escaped = True
                escape_time = t
                break
        
        states = np.array(states)
        return {
            'times': np.array(times),
            'positions': states[:, 0],
            'velocities': states[:, 1],
            'stores': states[:, 2],
            'escaped': escaped,
            'escape_time': escape_time
        }

# Test: Compare single frequency vs two sub-threshold frequencies
print("Testing multi-frequency driving...")
print("=" * 70)

# First, find the threshold with single frequency
multi_model = MultiFrequencyAtomModel(
    omega0=params.omega0, damping=params.damping,
    alpha=params.alpha, beta=params.beta
)

# Find single-frequency threshold
print("\n1. Finding single-frequency threshold...")
test_freqs = np.linspace(0.05, 0.3, 20)
single_results = []
for f in test_freqs:
    result = multi_model.simulate([f], [params.amplitude])
    single_results.append({'freq': f, 'escaped': result['escaped']})
    if result['escaped']:
        print(f"   f={f:.3f}: ESCAPED")
    else:
        print(f"   f={f:.3f}: bound")

# Find threshold
threshold_single = None
for i, r in enumerate(single_results):
    if r['escaped']:
        threshold_single = r['freq']
        break

if threshold_single:
    print(f"\n   Single-frequency threshold: ~{threshold_single:.3f}")
else:
    print("\n   No escape detected - adjusting parameters...")
    threshold_single = 0.15  # Use expected value


In [ ]:
# Test two sub-threshold frequencies together
print("\n2. Testing two sub-threshold frequencies together...")
print("-" * 70)

# Use frequencies below threshold
f_sub_1 = 0.08  # Below threshold
f_sub_2 = 0.09  # Below threshold
amp = params.amplitude

# Test each alone
result_f1_alone = multi_model.simulate([f_sub_1], [amp])
result_f2_alone = multi_model.simulate([f_sub_2], [amp])
result_combined = multi_model.simulate([f_sub_1, f_sub_2], [amp, amp])

print(f"\n   f1={f_sub_1} alone: {'ESCAPED' if result_f1_alone['escaped'] else 'bound'}")
print(f"   f2={f_sub_2} alone: {'ESCAPED' if result_f2_alone['escaped'] else 'bound'}")
print(f"   f1+f2 together: {'ESCAPED!' if result_combined['escaped'] else 'bound'}")

# Visualize
fig, axes = plt.subplots(2, 3, figsize=(15, 8))

# Row 1: Individual frequencies
ax1 = axes[0, 0]
ax1.plot(result_f1_alone['times'], result_f1_alone['positions'], 'b-', alpha=0.7)
ax1.set_title(f'f1={f_sub_1} alone\n{"ESCAPED" if result_f1_alone["escaped"] else "BOUND"}')
ax1.set_ylabel('Position x(t)')

ax2 = axes[0, 1]
ax2.plot(result_f2_alone['times'], result_f2_alone['positions'], 'g-', alpha=0.7)
ax2.set_title(f'f2={f_sub_2} alone\n{"ESCAPED" if result_f2_alone["escaped"] else "BOUND"}')

ax3 = axes[0, 2]
ax3.plot(result_combined['times'], result_combined['positions'], 'r-', alpha=0.7)
ax3.set_title(f'f1+f2 combined\n{"ESCAPED!" if result_combined["escaped"] else "BOUND"}')

# Row 2: Store dynamics
ax4 = axes[1, 0]
ax4.plot(result_f1_alone['times'], result_f1_alone['stores'], 'b-', alpha=0.7)
ax4.set_ylabel('Store s(t)')
ax4.set_xlabel('Time')

ax5 = axes[1, 1]
ax5.plot(result_f2_alone['times'], result_f2_alone['stores'], 'g-', alpha=0.7)
ax5.set_xlabel('Time')

ax6 = axes[1, 2]
ax6.plot(result_combined['times'], result_combined['stores'], 'r-', alpha=0.7)
ax6.set_xlabel('Time')

plt.suptitle('MULTI-FREQUENCY TEST: Can two sub-threshold frequencies cause escape?', 
             fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

# Key insight
print("\n" + "=" * 70)
print("MULTI-FREQUENCY PREDICTION:")
print("=" * 70)
if result_combined['escaped'] and not result_f1_alone['escaped'] and not result_f2_alone['escaped']:
    print("  ★ TWO SUB-THRESHOLD FREQUENCIES TOGETHER CAN CAUSE ESCAPE!")
    print("  ★ This is a testable prediction that differs from photon model!")
else:
    print("  The combined effect depends on parameters.")
    print("  Try adjusting f1, f2, or amplitudes to see combined effect.")
print("=" * 70)


## 9. Different Materials: Testing the Model

Different metals have different work functions (photoelectric thresholds). In the standard theory, this is because different materials have different binding energies.

In our model, different materials have different alpha, beta parameters. Let us see if our model can reproduce the correct ordering and approximate values!


In [ ]:
# Import material data and comparison functions
from src.physics_interpretation import (
    MATERIALS, MaterialProperties, 
    material_to_model_parameters, model_to_physical_threshold,
    compare_materials_threshold, explain_threshold_physics
)

# Print the threshold physics explanation
print(explain_threshold_physics())


In [ ]:
# Compare materials
print(compare_materials_threshold())

# Visualize material comparison
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

materials_data = []
for name, mat in MATERIALS.items():
    params_mat = material_to_model_parameters(mat)
    prediction = model_to_physical_threshold(params_mat)
    materials_data.append({
        'name': mat.name,
        'work_function': mat.work_function_eV,
        'actual_wavelength': mat.threshold_wavelength_nm(),
        'predicted_wavelength': prediction['threshold_wavelength_nm']
    })

# Sort by work function
materials_data.sort(key=lambda x: x['work_function'])

names = [m['name'] for m in materials_data]
work_functions = [m['work_function'] for m in materials_data]
actual = [m['actual_wavelength'] for m in materials_data]
predicted = [m['predicted_wavelength'] for m in materials_data]

# Left: Work function comparison
ax1 = axes[0]
x_pos = np.arange(len(names))
ax1.bar(x_pos - 0.2, work_functions, 0.4, label='Work Function (eV)', color='steelblue')
ax1.set_xticks(x_pos)
ax1.set_xticklabels([n.split('(')[0].strip() for n in names], rotation=45, ha='right')
ax1.set_ylabel('Work Function (eV)')
ax1.set_title('Work Functions of Different Materials')
ax1.grid(True, alpha=0.3, axis='y')

# Add values on bars
for i, v in enumerate(work_functions):
    ax1.text(i - 0.2, v + 0.1, f'{v:.2f}', ha='center', fontsize=9)

# Right: Threshold wavelength comparison
ax2 = axes[1]
ax2.bar(x_pos - 0.2, actual, 0.35, label='Actual', color='green', alpha=0.7)
ax2.bar(x_pos + 0.2, predicted, 0.35, label='Model Prediction', color='purple', alpha=0.7)
ax2.set_xticks(x_pos)
ax2.set_xticklabels([n.split('(')[0].strip() for n in names], rotation=45, ha='right')
ax2.set_ylabel('Threshold Wavelength (nm)')
ax2.set_title('Threshold Wavelength: Actual vs Model')
ax2.legend()
ax2.grid(True, alpha=0.3, axis='y')

plt.suptitle('Material Comparison: Can Our Model Predict Real Thresholds?', fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

print("\n" + "=" * 70)
print("KEY INSIGHT: Different materials have different thresholds because")
print("they have different internal dynamics (alpha, beta), not because")
print("light comes in discrete packets!")
print("=" * 70)


## 10. Testable Predictions: What Makes This Scientific

A model is only valuable if it makes TESTABLE PREDICTIONS that could be proven wrong. Our wave-only model makes several predictions that differ from the photon model!


In [ ]:
# Display testable predictions
print(get_testable_predictions())


## 11. Grand Summary: The Black Box Model

### What We Have Built:

**THE BLACK BOX**: A small region of crystal lattice that responds to incoming light waves. Inside, we have:
- Electron dynamics (position, velocity)
- Internal store/coherence mechanism
- Local fields
- Lattice vibrations
- Collective excitations

**THE EQUATIONS**: A coupled system of differential equations:
- Linear terms (harmonic oscillation, damping)
- Polynomial terms (f squared threshold)
- Product terms (coupling between subsystems)  
- Nonlinear terms (power absorption)

**THE KEY INSIGHT**: The frequency threshold emerges from RATE COMPETITION:
- Store charging rate proportional to f squared
- Store decay rate constant (alpha)
- Above threshold: charging wins leads to escape
- Below threshold: decay wins leads to bound state

**THE REVOLUTIONARY CLAIM**: The quantum may not be a property of LIGHT at all, but rather how MATTER organizes its internal degrees of freedom!


In [ ]:
# Final summary: The complete equation system
def print_complete_system():
    """Print the complete system of equations in a clear format."""
    
    print("=" * 80)
    print("      THE COMPLETE SYSTEM OF EQUATIONS FOR THE BLACK BOX MODEL")
    print("=" * 80)
    print("""
    STATE VECTOR: Y = [x, v, s, E_local, B_local, u_lattice, v_lattice, P, n_plasmon]
    
    ┌─────────────────────────────────────────────────────────────────────────────┐
    │  CORE ELECTRON DYNAMICS (The Minimal Model)                                 │
    ├─────────────────────────────────────────────────────────────────────────────┤
    │                                                                             │
    │    dx/dt = v                           [Kinematic]                          │
    │                                                                             │
    │    dv/dt = -ω₀²x - γv + s·u(t)         [Driven Oscillator + Amplification]  │
    │            ─────  ──   ─────                                                │
    │            binding damping  store-amplified drive                           │
    │                                                                             │
    │    ds/dt = -αs + β·f²·|v·u|            [Store Dynamics - THE KEY!]          │
    │            ──   ──────────                                                  │
    │            decay  frequency-dependent charging                              │
    │                                                                             │
    │    THRESHOLD CONDITION: f_threshold ~ √(α/β)                                │
    │                                                                             │
    └─────────────────────────────────────────────────────────────────────────────┘
    
    ┌─────────────────────────────────────────────────────────────────────────────┐
    │  EXTENDED MODEL (Full Crystal Lattice Black Box)                            │
    ├─────────────────────────────────────────────────────────────────────────────┤
    │                                                                             │
    │    dE_local/dt = -ωp²x - γp·E + source      [Local Field Evolution]         │
    │                                                                             │
    │    dB_local/dt = -∇×E                       [Magnetic Field]                │
    │                                                                             │
    │    du_L/dt = v_L                            [Lattice Position]              │
    │    dv_L/dt = -ωL²u_L - γL·v_L + coupling    [Lattice Vibrations]            │
    │                                                                             │
    │    dP/dt = ne·e·v - P/τ                     [Polarization]                  │
    │                                                                             │
    │    dn_pl/dt = generation - γ_pl·n_pl        [Plasmons]                      │
    │                                                                             │
    └─────────────────────────────────────────────────────────────────────────────┘
    
    ┌─────────────────────────────────────────────────────────────────────────────┐
    │  EQUATION TYPES IN THE SYSTEM                                               │
    ├─────────────────────────────────────────────────────────────────────────────┤
    │                                                                             │
    │    LINEAR:      dx/dt = v, du_L/dt = v_L                                    │
    │                                                                             │
    │    HARMONIC:    dv/dt = -ω²x - γv + ...                                     │
    │                                                                             │
    │    POLYNOMIAL:  ds/dt ~ f² (quadratic in frequency!)                        │
    │                                                                             │
    │    PRODUCT:     s·u, v·u (coupling between variables)                       │
    │                                                                             │
    │    NONLINEAR:   |v·u| (absolute value - rectification)                      │
    │                                                                             │
    └─────────────────────────────────────────────────────────────────────────────┘
    
    ┌─────────────────────────────────────────────────────────────────────────────┐
    │  PHYSICAL INTERPRETATION                                                    │
    ├─────────────────────────────────────────────────────────────────────────────┤
    │                                                                             │
    │    The f² term is the KEY to frequency threshold:                           │
    │                                                                             │
    │    • At low f: charging rate (βf²) < decay rate (α)                         │
    │                Store cannot maintain itself → electron bound                │
    │                                                                             │
    │    • At high f: charging rate (βf²) > decay rate (α)                        │
    │                 Store grows → amplification → escape!                       │
    │                                                                             │
    │    This creates photoelectric-like threshold from PURE WAVE MECHANICS!      │
    │                                                                             │
    └─────────────────────────────────────────────────────────────────────────────┘
    """)
    print("=" * 80)

print_complete_system()
print("\n✓ This notebook demonstrates that photoelectric threshold behavior")
print("  can emerge from classical wave mechanics with internal resonance!")
print("\n★ The 'quantum' might be in the MATTER, not the LIGHT! ★")


In [ ]:
# Bonus: Test the Compton scattering module (wave emission)
from src.compton import ComptonAtomModel, ComptonSimulation, analyze_compton_shift

print("=" * 70)
print("      COMPTON SCATTERING: Wave Emission and Frequency Shifts")
print("=" * 70)

# Create Compton model
compton_atom = ComptonAtomModel(
    omega0=1.0, damping=0.0001, alpha=0.05, beta=100.0, amplitude=0.2,
    emission_coupling=1.0, store_modulation=0.2
)

compton_sim = ComptonSimulation(compton_atom, t_max=50.0, dt=0.0005)

# Run for one frequency
test_freq = 0.2
compton_result = compton_sim.run(test_freq)

print(f"\nCompton-like scattering at f = {test_freq}:")
print(f"  Input frequency: {compton_result.input_frequency:.4f}")
print(f"  Output frequency: {compton_result.output_frequency:.4f}")
print(f"  Frequency shift: {compton_result.frequency_shift:.4f}")
print(f"  Energy ratio (out/in): {compton_result.energy_ratio:.4f}")

# Plot input vs output
fig, axes = plt.subplots(2, 2, figsize=(14, 8))

n_plot = min(1000, len(compton_result.times))

ax1 = axes[0, 0]
ax1.plot(compton_result.times[:n_plot], compton_result.input_wave[:n_plot], 'b-', alpha=0.7, label='Input')
ax1.set_title('Input Wave')
ax1.set_xlabel('Time')
ax1.legend()

ax2 = axes[0, 1]
ax2.plot(compton_result.times[:n_plot], compton_result.output_wave[:n_plot], 'r-', alpha=0.7, label='Output')
ax2.set_title('Output (Scattered) Wave')
ax2.set_xlabel('Time')
ax2.legend()

ax3 = axes[1, 0]
ax3.plot(compton_result.times[:n_plot], compton_result.positions[:n_plot], 'g-', alpha=0.7)
ax3.set_title('Electron Position')
ax3.set_xlabel('Time')

ax4 = axes[1, 1]
ax4.plot(compton_result.times[:n_plot], compton_result.stores[:n_plot], 'orange', alpha=0.7)
ax4.set_title('Internal Store')
ax4.set_xlabel('Time')

plt.suptitle(f'Compton-like Scattering: f_in={test_freq}, f_out={compton_result.output_frequency:.4f}', 
             fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

print("\n✓ The model shows wave emission from accelerating electron")
print("  This is the foundation for explaining Compton scattering with waves!")
